In [1]:
from package.database import get_clickhouse_client, render_statement
from projects.tutorial.config.settings import get_settings

settings = get_settings()

# Execute query

In [2]:
query = """
select name, engine
from system.databases
"""

with get_clickhouse_client(settings.db.URL) as client:
    df = client.query_df(query)

df

,name,engine
0,INFORMATION_SCHEMA,Memory
1,analytics,Atomic
2,default,Atomic
3,information_schema,Memory
4,system,Atomic


# Add parameters

In [3]:
query = """
select name, engine
from system.databases
where engine in {engines:Array(String)}
"""
parameters = {
    'engines': ['Atomic']
}

with get_clickhouse_client(settings.db.URL) as client:
    df = client.query_df(query, parameters=parameters)

df

,name,engine
0,analytics,Atomic
1,default,Atomic
2,system,Atomic


# Add Jinja

In [4]:
query = """
select {{ columns|join(', ') }}
from system.databases
where engine in {engines:Array(String)}
"""
context = {
    'columns': ['name', 'engine']
}
query = render_statement(query, context=context)

with get_clickhouse_client(settings.db.URL) as client:
    df = client.query_df(query, parameters=parameters)

df

,name,engine
0,analytics,Atomic
1,default,Atomic
2,system,Atomic
